# Building sequential model file

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

c:\users\sabik\envs\env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\sabik\envs\env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\sabik\envs\env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\sabik\envs\env\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
import pickle

pickle_in = open("../Dataset/X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("../Dataset/y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

In [3]:
from tensorflow.keras.callbacks import TensorBoard
import time
import datetime
import os

In [28]:
NAME = "math-symbol-CNN"
log_dir='.\logs\\fit\math-symbol-CNN8\\'
 
tensorboard = TensorBoard(log_dir)
print(log_dir)

.\logs\fit\math-symbol-CNN8\


In [29]:
X.shape[1:]

(28, 28, 1)

In [5]:
num_classes=26
def model1():
    model = Sequential()

    model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

    model.add(Dense(64))

    model.add(Dense(num_classes))
    model.add(Activation('sigmoid'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2)
    model.save('v1.model')
    
def model2():
    model = Sequential()
    model.add(Conv2D(32,(3,3), activation='relu',input_shape=X.shape[1:]))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(32,(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(64,activation='tanh'))
    model.add(Dropout(0.2))
    
    model.add(Dense(num_classes,activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(X, y, batch_size=32, epochs=15, validation_split=0.1,callbacks=[tensorboard])
    
model1()



W0919 10:17:17.169503  9800 deprecation.py:323] From c:\users\sabik\envs\env\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 166400 samples, validate on 41600 samples
Epoch 1/10
166400/166400 [==============================] - 104s 623us/sample - loss: 0.1516 - acc: 0.9576 - val_loss: 0.0625 - val_acc: 0.9823
Epoch 2/10
166400/166400 [==============================] - 102s 615us/sample - loss: 0.0493 - acc: 0.9860 - val_loss: 0.0517 - val_acc: 0.9866
Epoch 3/10
166400/166400 [==============================] - 102s 615us/sample - loss: 0.0323 - acc: 0.9907 - val_loss: 0.0439 - val_acc: 0.9886
Epoch 4/10
166400/166400 [==============================] - 102s 615us/sample - loss: 0.0240 - acc: 0.9930 - val_loss: 0.0411 - val_acc: 0.9900
Epoch 5/10
166400/166400 [==============================] - 102s 615us/sample - loss: 0.0184 - acc: 0.9947 - val_loss: 0.0424 - val_acc: 0.9900
Epoch 6/10
166400/166400 [==============================] - 102s 615us/sample - loss: 0.0157 - acc: 0.9956 - val_loss: 0.0394 - val_acc: 0.9909
Epoch 7/10
166400/166400 [==============================] - 101s 609us/sample - loss:

# tensorboard --logdir=ENTERLOGFOLDERPATH --host localhost --port 8088

In [24]:
import time
num_classes=26
results=[]
for conv2d_count in [1,2]:
    for dense_size in [128,256,512,1024,2048]:
        for dropout in [0.0,0.25,0.50,0.75]:
            model = Sequential()
            for i in range(conv2d_count):
                if i==0:
                    model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=X.shape[1:]))
                else:
                    model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
                model.add(MaxPooling2D(pool_size=(2,2)))
            model.add(Flatten())
            model.add(Dense(dense_size,activation='tanh'))
            if dropout>0.0:
                model.add(Dropout(dropout))
            model.add(Dense(num_classes,activation='softmax'))
            
            model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
            
            log_dir = '.\logsconv2d_%d-dense_%d-dropout_%.2f' % (conv2d_count,dense_size,dropout)
            tensorboard = TensorBoard(log_dir)
            
            start = time.time()
            
            model.fit(X,y,batch_size=32,epochs=10,verbose=0,validation_split=0.2,callbacks=[tensorboard])
#             score = model.evaluate(test_input,test_output,verbose=2)
            end=time.time()
            elapsed=end-start
            print('Conv2D count: %d, Dense size: %d, Dropout: %.2f , Time:%d sec' %  (conv2d_count, dense_size, dropout, elapsed ))

Conv2D count: 1, Dense size: 128, Dropout: 0.00 , Time:91 sec
Conv2D count: 1, Dense size: 128, Dropout: 0.25 , Time:95 sec


W0919 04:53:32.992662 12456 nn_ops.py:4224] Large dropout rate: 0.75 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Conv2D count: 1, Dense size: 128, Dropout: 0.50 , Time:95 sec
Conv2D count: 1, Dense size: 128, Dropout: 0.75 , Time:99 sec
Conv2D count: 1, Dense size: 256, Dropout: 0.00 , Time:104 sec
Conv2D count: 1, Dense size: 256, Dropout: 0.25 , Time:103 sec


W0919 05:00:27.006839 12456 nn_ops.py:4224] Large dropout rate: 0.75 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Conv2D count: 1, Dense size: 256, Dropout: 0.50 , Time:106 sec
Conv2D count: 1, Dense size: 256, Dropout: 0.75 , Time:104 sec
Conv2D count: 1, Dense size: 512, Dropout: 0.00 , Time:121 sec
Conv2D count: 1, Dense size: 512, Dropout: 0.25 , Time:124 sec


...

W0919 05:18:55.228289 12456 nn_ops.py:4224] Large dropout rate: 0.75 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Conv2D count: 1, Dense size: 1024, Dropout: 0.50 , Time:167 sec
Conv2D count: 1, Dense size: 1024, Dropout: 0.75 , Time:168 sec
Conv2D count: 1, Dense size: 2048, Dropout: 0.00 , Time:244 sec
Conv2D count: 1, Dense size: 2048, Dropout: 0.25 , Time:245 sec


W0919 05:34:01.135713 12456 nn_ops.py:4224] Large dropout rate: 0.75 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Conv2D count: 1, Dense size: 2048, Dropout: 0.50 , Time:246 sec


ResourceExhaustedError: OOM when allocating tensor with shape[5408,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node Adam_27/gradients/dense_54/MatMul_grad/MatMul_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


# Check prediction